In [1]:
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.3 MB/s eta 0:00:00
Mounted at /content/gdrive


In [2]:
from fastbook import *
from IPython.display import display,HTML

In [3]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)
#if __name__ == "__main__":
#    dls = TextDataLoaders.from_folder(path, valid='test', bs=16)


In [4]:
#path = untar_data(URLs.IMDB)
#dls = DataBlock(
#    blocks=(TextBlock.from_folder(path),CategoryBlock),
#    get_y = parent_label,
#    get_items = partial(get_text_files, folders=['train', 'test']),
#    splitter = GrandparentSplitter(valid_name='test')
#).dataloaders(path)

In [5]:
path = untar_data(URLs.IMDB)

files = get_text_files(path,folders=['train','test'])
txts = L(o.open().read() for o in files[:2000])

In [6]:
spacy = WordTokenizer()
tkn = Tokenizer(spacy)

In [7]:
tkn.setup(txts)

tokns = txts.map(tkn)

In [8]:
tokns[0]

(#171) ['xxbos','xxmaj','this','was','really','a','very','bad','movie','.'...]

In [9]:
num = Numericalize()
num.setup(tokns)
nums = tokns.map(num)
nums[0][:10]

TensorText([ 2,  8, 20, 24, 79, 12, 88, 72, 27, 10])

In [10]:
nums_dec = num.decode(nums[0][:10]);
nums_dec

(#10) ['xxbos','xxmaj','this','was','really','a','very','bad','movie','.']

In [11]:
tkn.decode(nums_dec)

'xxbos xxmaj this was really a very bad movie .'

In [12]:
tkn((txts[0], txts[1]))

((#171) ['xxbos','xxmaj','this','was','really','a','very','bad','movie','.'...],
 (#146) ['xxbos','i','voted','3','for','this','movie','because','it','looks'...])

In [13]:
def f(x:int): return x+1

tfm = Transform(f)
tfm(2), tfm(2.0)

(3, 2.0)

In [14]:
@Transform
def f(x:int): return x+1

f(2), f(2.0)

(3, 2.0)

In [15]:
class NormalizeMean(Transform):
  def setups(self,x): self.mean = sum(x)/len(x)
  def encodes(self,x): return x-self.mean
  def decodes(self,x): return x + self.mean

In [16]:
tfm = NormalizeMean()
tfm.setups([1,2,3,4,5])
start = 2
y = tfm(start)
print(y)
z = tfm.decode(y)
tfm.mean, y,z

-1.0


(3.0, -1.0, 2.0)

In [17]:
tfms= Pipeline([tkn, num])
t = tfms(txts[0]);
t[:20]

TensorText([   2,    8,   20,   24,   79,   12,   88,   72,   27,   10,   18,  249,   12,  683,  383,   15,    8, 1394,    8,  185])

In [18]:
tfms.decode(t)

"xxbos xxmaj this was really a very bad movie . i am a huge fan of xxmaj italian xxmaj horror , xxmaj argento , xxmaj mario xxmaj xxunk , xxmaj fulci and yes , even our good friend here xxmaj xxunk sometimes comes out with a good one . i found the first two ' demons ' films to be highly entertaining - they were so bad they were great but this one is just so bad that it is really , really bad . xxmaj it is intensely boring , the story never goes anywhere and i hated the characters - the wife slapping husband and whiny cry - baby pain in the xxrep 3 * wife drove me mad , there was nowhere near enough of the story devoted to the xxmaj xxunk who was probably the best actor in the whole film . i turned it off about three xxunk of the way through because i was very , very xxup bored ! xxmaj do n't bother ."

# TfmdLists

In [20]:
tls = TfmdLists(files, [tkn,num])

In [21]:
t = tls[0]; t[:20]

TensorText([   2,    8,   20,   24,   79,   12,   88,   72,   27,   10,   18,  249,   12,  683,  383,   15,    8, 1394,    8,  185])

In [22]:
tls.decode(t)[:100]

'xxbos xxmaj this was really a very bad movie . i am a huge fan of xxmaj italian xxmaj horror , xxmaj'

In [23]:
tls.show(t)

xxbos xxmaj this was really a very bad movie . i am a huge fan of xxmaj italian xxmaj horror , xxmaj argento , xxmaj mario xxmaj xxunk , xxmaj fulci and yes , even our good friend here xxmaj xxunk sometimes comes out with a good one . i found the first two ' demons ' films to be highly entertaining - they were so bad they were great but this one is just so bad that it is really , really bad . xxmaj it is intensely boring , the story never goes anywhere and i hated the characters - the wife slapping husband and whiny cry - baby pain in the xxrep 3 * wife drove me mad , there was nowhere near enough of the story devoted to the xxmaj xxunk who was probably the best actor in the whole film . i turned it off about three xxunk of the way through because i was very , very xxup bored ! xxmaj do n't bother .


In [24]:
cut = int(len(files) * 0.8)
splits = [list(range(cut)), list(range(cut,len(files)))]

tls = TfmdLists(files, [tkn,num], splits = splits)

In [25]:
tls.valid[0][:20]

TensorText([   2,    8,  108,   11,    8,   18,  149,   53,  136,    8, 4077,    8, 2005,   23,   33, 2433,   19,    8,    0,    8])

In [26]:
tls.decode(tls.valid[0][:20])

"xxbos xxmaj well , xxmaj i 've just seen xxmaj buster xxmaj keaton 's film debut in xxmaj xxunk xxmaj"

In [27]:
lbls = files.map(parent_label)
lbls

(#50000) ['neg','neg','neg','neg','neg','neg','neg','neg','neg','neg'...]

In [32]:
cat = Categorize()
cat.setup(lbls)
cat.vocab, cat(lbls[20000])

(['neg', 'pos'], TensorCategory(1))

In [34]:
tls_y = TfmdLists(files, [parent_label, Categorize()])
tls_y[0]

TensorCategory(0)

# Datasets
This is being done to keep independent and dependent variables together

In [35]:
x_tfms = [tkn,num]
y_tfms = [parent_label,Categorize()]

dsets = Datasets(files, [x_tfms,y_tfms])
x,y=dsets[0]
x,y


(TensorText([   2,    8,   20,   24,   79,   12,   88,   72,   27,   10,   18,  249,   12,  683,  383,   15,    8, 1394,    8,  185,   11,    8, 6575,   11,    8, 7910,    8,    0,   11,    8, 2370,
               13,  451,   11,   68,  351,   70,  533,  154,    8,    0,  612,  294,   62,   30,   12,   70,   43,   10,   18,  287,    9,  109,  140,   63, 5028,   63,  137,   14,   40,  950,  474,
               25,   44,   80,   48,   72,   44,   80,  166,   29,   20,   43,   16,   53,   48,   72,   21,   17,   16,   79,   11,   79,   72,   10,    8,   17,   16, 7911,  245,   11,    9,   92,
              126,  285, 1121,   13,   18, 1002,    9,  122,   25,    9,  413, 4046,  689,   13, 5698, 1838,   25, 1073, 1909,   19,    9,    5,  134,  186,  413, 2966,   90, 1306,   11,   50,   24,
              870,  672,  204,   15,    9,   92, 3692,   14,    9,    8,    0,   59,   24,  263,    9,  208,  280,   19,    9,  206,   33,   10,   18,  664,   17,  129,   61,  332,    0,   15,    9,
     

In [38]:
# along with splits

x_tfms = [tkn,num]
y_tfms = [parent_label, Categorize()]

dsets = Datasets(files, [x_tfms,y_tfms], splits=splits)
x,y = dsets.train[10]
x,y

(TensorText([   2,    8,   20,   27,   24,   53,  210,  234,   72,   10,   18,  171,  588,  112,   13,   84, 1469,  241,  255,   58,  114,  112,   13,  209,  158,   21,   18,  589,   11,   29,   20,
               24,   36,   43,   15,  110,   10,    8,   20,   27, 4056, 1912,   13,    0,   18,  101,   17,   11,    9,    8,    0,  312,   75,   43,  491,   15,   12,  559,   13,   21,  546,  166,
               11,   29,    9,   92,   16,  580,  643,   10,    8,   34,   52,   32,   39,  106,  179, 1993,   30,  106,   15,    9,  122,   13,   50,   23,   67,  179,   92,  363,   14,  936,   10,
                8,   34,   53,  150,   58,   43, 1055,  156,   14,  183,   11,  145,    0,   14,  807,    9,   92,   21,   16,  251,   14,   40, 7933,   10,    8,   60,   34,  194,   12,  588,   27,
               21,  118,  445,   34,    0,   11,   13, 2382,  204,  238, 2808,  152,   11,  103,   18,   69, 1171,   22, 6595,   22,   41, 4057,    8, 1311,   33,   37,   10,    8,   55,   60,   34,
     

In [39]:
t = dsets.valid[0]
dsets.decode(t)

("xxbos xxmaj well , xxmaj i 've just seen xxmaj buster xxmaj keaton 's film debut in xxmaj xxunk xxmaj xxunk 's xxmaj the xxmaj butcher xxmaj boy and - despite the crude way everything just seems to happen for almost no logical reason - i found plenty to laugh at . xxmaj like when xxmaj buster orders molasses from butcher boy xxmaj xxunk , xxmaj xxunk makes xxmaj buster come back to pay , xxmaj buster says he put it in the bucket that has the molasses , xxmaj xxunk xxunk molasses in xxmaj buster 's hat and takes money , xxmaj buster takes hat back on head as it gets stuck , xxmaj xxunk attempts to remove it while molasses fall to floor , xxmaj buster 's feet are now stuck on floor and so on . xxmaj that probably did n't read funny but on screen it was hilarious as were some more slapstick involving xxunk being thrown and a later sequence that takes place in xxmaj xxunk 's girlfriend 's xxunk school with xxmaj xxunk dressed in drag and xxmaj buster helping xxmaj xxunk 's rival also in 

In [40]:
dls = dsets.dataloaders(bs=64,  before_batch = pad_input)

In [ ]:
# final code

tfms = [[tkn,num], [parent_label,Categorize()]]
files = get_text_files(path,folders=['train','test'])
splits = GrandparentSplitter(valid_name='test')(files)
dsets = Datasets(files,tfms,splits=splits)

dls = dsets.dataloaders(dl_type=SortedDL,before_batch=pad_input)

## This was the way to customize the transform steps every single piece.